In [5]:
#import voor alle benodigde libraries
import requests
import pandas as pd
import json
from sodapy import Socrata

In [6]:
#Inladen API chargemap
response = requests.get("https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&maxresults=8000&compact=true&verbose=false&key=6686970f-0aa6-4cb5-ae81-cf8f6389a816")

#Omzetten naar dictionary
responsejson  = response.json()

#Dataframe bevat kolom die een list zijn. 
#Met json_normalize zet je de eerste kolom om naar losse kolommen
charge_map = pd.json_normalize(responsejson)

charge_map_df = pd.DataFrame.from_records(charge_map)

#Dropping unnecessary columns:
charge_map_df = charge_map_df.drop(columns=['IsRecentlyVerified', 'UUID', 'DataProviderID', 'AddressInfo.CountryID', 
                            'AddressInfo.DistanceUnit', 'AddressInfo.RelatedURL', 'AddressInfo.ContactTelephone1', 
                            'AddressInfo.ContactEmail', 'AddressInfo.ContactTelephone2', 'OperatorsReference', 
                            'DataProvidersReference', 'GeneralComments', 'AddressInfo.AddressLine2',
                            'AddressInfo.AccessComments'
                            ])

#Renaming column names:
charge_map_df.rename(columns={'AddressInfo.Latitude':'LAT', 'AddressInfo.Longitude':'LON', 
                              'AddressInfo.Postcode':'Postcode', 'AddressInfo.ID':'Address_ID', 
                              'AddressInfo.Title':'Address_Title', 'AddressInfo.AddressLine1':'Address_Line1',
                              'AddressInfo.StateOrProvince':'Address_StateOrProvince', 
                              'AddressInfo.Town':'Address_Town'},
                              inplace=True)                            


In [7]:
charge_map_df.describe()
#We zien dat LAT een te lage min value heeft, outliers die gooien we eruit

,ID,OperatorID,UsageTypeID,NumberOfPoints,StatusTypeID,DataQualityLevel,SubmissionStatusTypeID,Address_ID,LAT,LON
count,7898.000000,7535.000000,799.000000,7897.000000,7898.000000,7897.000000,7898.000000,7898.000000,7898.000000,7898.000000
mean,50502.966700,140.614068,3.002503,1.437255,50.026589,2.834874,108.546467,50847.563054,52.075229,5.121947
std,27214.344089,231.152105,1.909610,1.832636,2.356276,0.550476,27.958963,27218.821581,0.546326,0.694905
min,4621.000000,1.000000,0.000000,0.000000,0.000000,1.000000,100.000000,4566.000000,24.000000,3.382141
25%,35353.250000,73.000000,1.000000,1.000000,50.000000,3.000000,100.000000,35699.250000,51.836823,4.584757
50%,38642.500000,157.000000,4.000000,1.000000,50.000000,3.000000,100.000000,38988.500000,52.068354,5.005548
75%,63364.750000,178.000000,4.000000,2.000000,50.000000,3.000000,100.000000,63710.750000,52.321805,5.612725
max,258262.000000,3574.000000,7.000000,72.000000,210.000000,3.000000,200.000000,258646.000000,53.478088,7.189636


In [8]:
#Drop de outlier
charge_map_df = charge_map_df.drop(charge_map_df[charge_map_df['LAT'] < 50].index)

In [9]:
charge_map_df.isna().sum()
#We zien dat UsageTypeID en Address_StateOrProvince meer dan 80% NaN hebben, die filteren we eruit.
charge_map_df = charge_map_df.drop(columns=['UsageTypeID', 'Address_StateOrProvince'])

In [10]:
charge_map_df.head()

,DateLastVerified,ID,OperatorID,Connections,NumberOfPoints,StatusTypeID,DateLastStatusUpdate,DataQualityLevel,DateCreated,SubmissionStatusTypeID,Address_ID,Address_Title,Address_Line1,Address_Town,Postcode,LAT,LON,UsageCost,MetadataValues,DateLastConfirmed
0,2023-03-21T12:10:00Z,258262,103.0,"[{'ID': 431516, 'ConnectionTypeID': 33, 'Statu...",1.0,50,2023-03-21T12:10:00Z,1.0,2023-03-21T09:35:00Z,200,258646,Groene Kruisweg,Groene Kruisweg,Spijkenisse,3208 KA,51.858566,4.293632,NaN,NaN,NaN
1,2023-03-15T11:19:00Z,256957,1.0,[],1.0,50,2023-03-15T11:19:00Z,1.0,2023-03-15T10:20:00Z,200,257341,De Wildkansel,De Wildkansel,NaN,8251 ME,52.521086,5.731164,NaN,NaN,NaN
2,2023-03-04T15:56:00Z,238390,47.0,"[{'ID': 392175, 'ConnectionTypeID': 33, 'Statu...",1.0,50,2023-03-04T15:56:00Z,1.0,2023-03-04T15:56:00Z,200,238774,Mc Donalds Ijsselstein,Lorentzlaan,IJsselstein,3401 MX,52.014896,5.062458,NaN,NaN,NaN
3,2023-03-04T15:52:00Z,238389,47.0,"[{'ID': 392173, 'ConnectionTypeID': 33, 'Statu...",6.0,50,2023-03-04T15:52:00Z,1.0,2023-03-04T15:52:00Z,200,238773,Haje Allego Ultra E Nieuwegein Oost,Waterliniedok,Nieuwegein,3433 NV,52.011182,5.122047,NaN,NaN,NaN
4,2023-03-04T15:49:00Z,238388,47.0,"[{'ID': 392171, 'ConnectionTypeID': 33, 'Statu...",1.0,50,2023-03-04T15:49:00Z,1.0,2023-03-04T15:49:00Z,200,238772,Mc Donalds Nieuwegein,Ravenswade,Nieuwegein,3439 LD,52.049201,5.117389,NaN,NaN,NaN


In [11]:
#We hebben bepaald dat de data in de volgende columns niet nodig gaat zijn:
charge_map_df = charge_map_df.drop(columns=['OperatorID', 'Connections', 'StatusTypeID', 'SubmissionStatusTypeID'])

In [12]:
#Dropping empty values
charge_map_df = charge_map_df.drop(charge_map_df[charge_map_df['Postcode']==''].index)
charge_map_df = charge_map_df.drop(charge_map_df[charge_map_df['Postcode']=='XG'].index)
charge_map_df = charge_map_df.dropna(subset=['Postcode'])

#Transforming the Postcode to Integers
charge_map_df['Postcode'] = charge_map_df['Postcode'].astype(str)
charge_map_df['Postcode'] = charge_map_df['Postcode'].str[0:4]
charge_map_df['Postcode'] = charge_map_df['Postcode'].astype(int)

#Dropping incorrect postcodes
charge_map_df = charge_map_df.drop(charge_map_df[charge_map_df['Postcode'] < 1000].index)

In [13]:
#Creating address types to make color coding possible.
#Observerd types:
#-Restaurants   = MCDonald's / Mc Donald's,
#-Recreatie     = hotel, Gelredome
#-Parkeren      = P+R, parkeerterrein
#-Tankstations  = Fastned, Supercharger (Tesla),

#Recreation type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Hotel', case=False), 'Address_Type'] = 'Recreatie'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Gelredome', case=False), 'Address_Type'] = 'Recreatie'

#Tankstation type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Fastned', case=False), 'Address_Type'] = 'Tankstation'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Supercharger', case=False), 'Address_Type'] = 'Tankstation'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Shell', case=False), 'Address_Type'] = 'Tankstation'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('IONITY', case=False), 'Address_Type'] = 'Tankstation'

#Restaurants type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Mc Donald', case=False), 'Address_Type'] = 'Restaurant'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('McDonald', case=False), 'Address_Type'] = 'Restaurant'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Restaurant', case=False), 'Address_Type'] = 'Restaurant'

#Parkeerplaats type
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Parkeer', case=False), 'Address_Type'] = 'Parkeerterrein'
charge_map_df.loc[charge_map_df['Address_Title'].str.contains('Garage', case=False), 'Address_Type'] = 'Parkeerterrein'

#Filling in the NaNs with 'Straat'
charge_map_df['Address_Type'].fillna('Straat', inplace=True)

In [14]:
#Dropping typos
#De typos zijn handmatig bepaald door te kijken naar de indexen van outliers in de folium map
typos = [364, 4947, 1414, 1203, 5479, 1351, 2387, 1242, 479, 1882, 1597, 2413, 2064, 1670, 1941, 1982, 4094, 2831, 7198]
charge_map_df = charge_map_df.drop(index= typos)

In [15]:
#Changing the column 'NumberOfPoints' to string
charge_map_df['NumberOfPoints'] = charge_map_df['NumberOfPoints'].astype(int).astype(str)


In [16]:
#Het downloaden van de schone CSV
charge_map_df.to_csv(r'C:\School\Y3\Semester 2 - Minor\Block 1\Case3\chargemap.csv')

print('File has been saved')

File has been saved
